### import required module

In [2]:
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import os
from collections import defaultdict
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import scipy.sparse as sp

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:472: FutureWarning: Passi

### モデルを復元

In [3]:
# model_file = 'model'
# meta_file_name = 'weights-26.meta'
model_file = 'model_last-fm_epoch=300'
meta_file_name = 'weights-299.meta'

In [4]:
sess = tf.Session()
saver = tf.train.import_meta_graph('./{}/{}'.format(model_file, meta_file_name))

# モデルの復元
saver.restore(sess,tf.train.latest_checkpoint('./' + model_file + '/'))
# グラフを復元
graph = tf.get_default_graph()

INFO:tensorflow:Restoring parameters from ./model_last-fm_epoch=300/weights-299


In [5]:
user_embed = graph.get_tensor_by_name("user_embed:0")
entity_embed = graph.get_tensor_by_name("entity_embed:0")
relation_embed = graph.get_tensor_by_name("relation_embed:0")

### 予測結果をロード

In [6]:
saved_path = '../Analysis/each_user_result/'
file_name = 'last-fm_each_user_ret_epoch=0.pickle'

In [7]:
with open (saved_path + file_name, 'rb') as f:
    result = pickle.load(f)

print(result[0])

{'user_id': 0, 'recall': array([0.08510638, 0.12765957, 0.14893617, 0.14893617, 0.14893617]), 'precision': array([0.2       , 0.15      , 0.11666667, 0.0875    , 0.07      ]), 'ndcg': array([0.34691272, 0.4642606 , 0.51347782, 0.51347782, 0.51347782]), 'hit_ratio': array([1., 1., 1., 1., 1.]), 'auc': 0.0, 'top_N_items': [(1085, 11.455713), (10778, 11.330286), (23493, 11.135474), (11184, 11.129554), (11211, 10.938282), (8523, 10.843382), (2237, 10.800094), (8422, 10.597556), (10936, 10.547527), (9491, 10.531397), (19439, 10.492456), (10874, 10.43594), (9454, 10.372491), (9492, 10.35069), (14037, 10.296987), (7140, 10.198753), (23496, 10.131675), (7536, 10.083475), (8524, 9.937377), (9493, 9.814255)]}


In [8]:
user_rec_items = defaultdict()

for user, ret in result.items():
    items = dict(ret['top_N_items'])
    items = items.keys()
    
    user_rec_items[user]  = list(items)

### Attention Score のロード

In [9]:
saved_path = '../RestoreModel/attention_score/'
file_name = 'attention_score_epoch=19.pickle'

with open (saved_path + file_name, 'rb') as f:
    attention_score = pickle.load(f)

### Attention Score の算出

In [27]:
kge_dim = 64

def _generate_transE_score( h, t, r):
        
        embeddings = tf.concat([user_embed, entity_embed], axis=0)
        embeddings = tf.expand_dims(embeddings, 1)

        h_e = tf.nn.embedding_lookup(embeddings, h)
        t_e = tf.nn.embedding_lookup(embeddings, t)
        
        # relation embeddings: batch_size * kge_dim
        r_e = tf.nn.embedding_lookup(relation_embed, r)

    
        # relation transform weights: batch_size * kge_dim * emb_dim
        trans_M = tf.nn.embedding_lookup(trans_W, r)

        # batch_size * 1 * kge_dim -> batch_size * kge_dim
        h_e = tf.reshape(tf.matmul(h_e, trans_M), [-1, self.kge_dim])
        t_e = tf.reshape(tf.matmul(t_e, trans_M), [-1, self.kge_dim])

        # l2-normalize
        h_e = tf.math.l2_normalize(h_e, axis=1)
        r_e = tf.math.l2_normalize(r_e, axis=1)
        t_e = tf.math.l2_normalize(t_e, axis=1)

        kg_score = tf.reduce_sum(tf.multiply(t_e, tf.tanh(h_e + r_e)), 1)

        return kg_score

In [28]:
kg_score = _generate_transE_score(1,2,3)

NameError: name 'trans_W' is not defined

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    sess.run(kg_score)
    kg_score = kg_score.eval()

In [ ]:
kg_score

In [ ]:
% conda list